<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/ThomasSimonini%2FUnit3/notebooks/unit3/unit3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 3: 用 RL Baselines3 Zoo 进行深度 Q 学习来玩 Atari 游戏 👾  

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/thumbnail.jpg" alt="Unit 3 Thumbnail">

在 Notebook 中，你将使用 [RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) ,一个基于 [Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/) 的训练框架其可以使用脚本去训练，评估调整超参数图示结果和视频记录，训练一个深度 Q 学习智能体去玩 Space Invaders 游戏。

我们使用 [RL-Baselines-3 Zoo 集成, 深度 Q 学习的原始版本](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) 其不包含像双深度 Q 网络 ，竞争深度 Q 网络或者优先级经验回放的扩充。

⬇️ 这是你将要**达成效果**的例子 ⬇️ence Replay.


In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/ThomasSimonini/ppo-SpaceInvadersNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

### 🎮 环境：

- SpacesInvadersNoFrameskip-v4 

### 📚 RL-库:

- [RL-Baselines3-Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

## 目标 🏆

在本笔记结束，你将会得到：

- 能够对  **RL Baselines3 Zoo 怎么运作**有更深的理解。
- 能够把你训练的智能体和代码推送到 Hub 上并展示一个不错的视频效果和评估分数 🔥。





## 这本 notebook 来自深度强化学习课程
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

在这门免费的课程中，你将会：

- 📖 学习深度强化学习的**理论和实践**。
- 🧑‍💻 学习使用一些著名的**深度强化学习库**，比如 Stable Baselines3、RL Baselines3 Zoo、CleanRL 和 Sample Factory 2.0。
- 🤖 在**独特的环境**中训练智能体
更多详情请查看课程大纲 👉 https://simoninithomas.github.io/deep-rl-course

别忘了**<a href="http://eepurl.com/ic5ZUD">报名参加课程</a>** （我们收集你的电子邮件以便在每个单元发布时向你**发送链接并提供有关挑战和更新的信息**）。

保持联系的最佳方式是加入我们的 Discord 服务器与社区和我们交流 👉🏻 https://discord.gg/ydHrjt3WP5

## 前置准备 🏗️
在进入本笔记前，你需要：

🔲 📚 **[学习 Unit 3 的深度 Q 学习 ](https://huggingface.co/deep-rl-course/unit3/introduction)**  🤗


我们会持续提升我们的教程质量，如果你在本笔记中发现了什么问题请 [在 Github Repo 上提出 issue](https://github.com/huggingface/deep-rl-class/issues)。

# 让我们训练一个深度 Q 学习智能体去玩 Atari  Space Invaders  👾 并把他上传到 Hub。

为了验证这个实践项目是否通过认证流程，你需要将训练好的模型推送到 Hub 并且获得大于**等于200的结果**。

要查找你的结果，请转到排行榜并找到您的模型，**结果=平均回报-回报标准差**。

有关认证流程的更多信息，请查看此部分👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## 一条建议 💡
最好在你的 Google Drive 上复制这个 Colab，这样，如果**它超时**，你仍然可以在你的 Google Drive 上保存笔记本，并且不需要重新填写所有内容。

要这样做，你可以使用 `Ctrl + S` 或 `File > Save a copy in Google Drive`保存副本。

另外，我们将用 **1M 个时间步长训练 90 分钟**。通过键入 `!nvidia-smi`，你可以查看你正在使用哪个 GPU。

如果你想训练更多的 1000 万个步骤，这将需要大约 9 小时，有可能导致 Colab 超时。在这种情况下，建议在本地计算机上（或其他地方）运行。只需点击：`File>Download`。

## 设置 GPU 💪

- 为了 **加速智能体的训练，我们将使用 GPU**. 点击 `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 创建一个虚拟显示 🔽

在 Notebook 中, 我们需要生成一个回放视频。在 colab 中，**我们需要一个虚拟屏幕去渲染环境**（并记录帧）。

所以参照下面单元格安装相关库并运行虚拟屏幕 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Additional dependencies for RL Baselines3 Zoo
!apt-get install swig cmake freeglut3-dev 

In [ ]:
!pip install pyglet==1.5.1

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## 拷贝 RL-Baselines3 Zoo 仓库 📚
你可以直接从 Python 包 (`pip install rl_zoo3`) 安装,但是想要**带有额外环境和依赖的全安装**，我们需要拷贝 `RL-Baselines3-Zoo` 仓库并从源码安装。


In [ ]:
!git clone https://github.com/DLR-RM/rl-baselines3-zoo

## 安装依赖 🔽
我们现在可以安装 RL-Baselines3 Zoo 需要的依赖 (这大概持续 5min ⏲)

In [ ]:
%cd /content/rl-baselines3-zoo/

In [ ]:
!pip install -r requirements.txt

## 训练我们的深度 Q 学习智能体去玩 Space Invaders 👾

要用 RL-Baselines3-Zoo 训练智能体，我们仅需做两件事：
1. 在 `rl-baselines3-zoo/hyperparams/dqn.yml` 里定义超参数
   
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/hyperparameters.png" alt="DQN Hyperparameters">


我们可以看到：
- 使用 `Atari Wrapper` 进行预加载 (缩减帧, 灰度, 堆叠四帧),
- 使用 `CnnPolicy`, 用卷积层加载帧。
- 我们用一千万 `n_timesteps` 训练模型。
- 内存大小 (经验回放) 大小是 100000 ,也就是你保存的经验步数量再次训练智能体。

💡 我的建议是**减少训练的时间步到 1M**，这大概在 P100 的机器上花 90min。 `!nvidia-smi` 命令会显示你的 GPU 运行情况。在一千万步下,他将花 9 个小时 ，这有可能导致 Coblab 超时。这里建议你在你本地的机子（或其他）上面跑代码，只需要点击： `File>Download`。

为了优化超参数，我的建议是聚焦下面三个超参数：
- `learning_rate`
- `buffer_size (Experience Memory size)`
- `batch_size`

经验告诉我们, 你需要**检查文档并理解每个超参数的意义**： https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters



2. 运行 `train.py` 并把模型保存在 `logs` 文件夹 📁

In [ ]:
!python train.py --algo ________ --env SpaceInvadersNoFrameskip-v4  -f _________

#### 参考答案

In [ ]:
!python train.py --algo dqn  --env SpaceInvadersNoFrameskip-v4 -f logs/

## 让我们评估我们的智能体 👀
- RL-Baselines3-Zoo 提供 `enjoy.py` 文件, 一个评估智能体的 python 脚本。在大多数 RL 库中，我们都把评估脚本叫做 `enjoy.py`.
- 让我们用 5000 步评估一下  🔥

In [ ]:
!python enjoy.py  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps _________  --folder logs/


#### 参考答案

In [ ]:
!python enjoy.py  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps 5000  --folder logs/

## 把我们训练的模型发布在 Hub 上 🚀
现在经过训练后我们得到了不错的分数，我们可以通过以下一行代码将我们训练的模型发布到 Hub 🤗上。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/space-invaders-model.gif" alt="Space Invaders model">

使用 `rl_zoo3.push_to_hub.py`, **你评估记录重演，生成了你智能体的模型卡，并把他推送到了 Hub**。

这边：
- 你可以 **展示我们的工作** 🔥
- 你可以 **可视化你的智能体运行** 👀
- 你可以 **在社区分享一个他人可用的智能体** 💾
- 你可以 **获取 leaderboard 🏆 和你的同学对比查看你的智能体的表现性能** 👉  https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

为了将你的模型分享到社区，你需要遵循下面步骤：

1️⃣ (如果还没做) 在 HF 创建一个账号 ➡ https://huggingface.co/join

2️⃣ 登录 HF 获取你的认证 token 。
- 创建**带有写权限**的新的 token (https://huggingface.co/settings/tokens) 

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- 复制 token
- 运行下面单元格并粘贴 token

In [ ]:
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
notebook_login()
!git config --global credential.helper store

如果你不想使用 Google Colab 或者 Jupyter Notebook，你需要用这个命令代替：`huggingface-cli login`

3️⃣ 我们现在已经准备好推送你的智能体到 Hub 🤗了 🔥

让我们运行 `push_to_hub.py` 去上传我们训练的智能体到 Hub。这里有两个重要参数:

* `--repo-name `: 仓库名
* `-orga`: 你的 Hugging Face 用户名

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/select-id.png" alt="Select Id">

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name _____________________ -orga _____________________ -f logs/


#### 参考答案

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name dqn-SpaceInvadersNoFrameskip-v4  -orga ThomasSimonini  -f logs/

恭喜 🥳 你已经完成训练并上传了你的第一个用 RL-Baselines-3 Zoo 的深度 Q 学习智能体。上面的脚本已经展示了一个模型仓库的链接像 https://huggingface.co/ThomasSimonini/dqn-SpaceInvadersNoFrameskip-v4。当你点击这个链接，你可以：

- 在右边看见你**智能体的预览**。
- 点击 "Files and versions" 查看所有仓库中的文件。
- 点击 "Use in stable-baselines3" 获取怎么加载模型的代码片段。
- 模型卡 (`README.md` 文件) 描述了模型和你使用的超参数。

底层看，Hub 使用基于 git 的仓库（如果你不知道啥是 git ，别担心）。这意味这你可以更新版本并提升你的智能体模型。

**和你的同伴比较一下结果** 使用 [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) 🏆

## 加载一个优秀的训练好的模型 🔥

Stable-Baselines3 团队在 Hub 上**上传了超过 150 给训练好的深度强化学习智能体**。你可以下载并观察他们怎么运行！

你可以在这里找到他们：👉 https://huggingface.co/sb3

一些例子：
- Asteroids: https://huggingface.co/sb3/dqn-AsteroidsNoFrameskip-v4
- Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4
- Breakout: https://huggingface.co/sb3/dqn-BreakoutNoFrameskip-v4
- Road Runner: https://huggingface.co/sb3/dqn-RoadRunnerNoFrameskip-v4

让我们加载一个智能体去玩 Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4


In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

1. 我们使用 `rl_zoo3.load_from_hub` 下载模型, 然后把他放到 `rl_trained` 的新文件夹里

In [ ]:
# Download model and save it into the logs/ folder
!python -m rl_zoo3.load_from_hub --algo dqn --env BeamRiderNoFrameskip-v4 -orga sb3 -f rl_trained/

2. 让我们运行 5000 步评估一下

In [ ]:
!python enjoy.py --algo dqn --env BeamRiderNoFrameskip-v4 -n 5000  -f rl_trained/

为啥不试着训练一下你自己的**深度 Q 学习智能体去玩 BeamRiderNoFrameskip-v4? 🏆.**

如果你想尝试, 查看链接 https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4#hyperparameters. 这里, **在模型卡里，你会获得训练好的智能体超参数。**

但是找到超参数是一件艰巨的事情，幸运的是，我们将会在下一个奖励单元中展示**怎样使用 Optuna 优化你的超参数 🔥。**

## 一些额外的挑战 🏆

最佳的学习方式就是**亲自尝试**！

在 [Leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) 你将会找到你的智能体。你能打榜登顶吗？

这里有一份环境列表你可以尝试训练你的智能体：
- BeamRiderNoFrameskip-v4
- BreakoutNoFrameskip-v4
- EnduroNoFrameskip-v4
- PongNoFrameskip-v4

同样，**如果你想要学习自己执行深度 Q 学习**,你一定要看一看 CleanRL 运行过程 https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/dqn_atari.py

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/atari-envs.gif" alt="Environments"/>

________________________________________________________________________
恭喜你完成本章!

如果你仍旧对某些部分感到困惑...完全正常！**这对于所有学习 RL 的人都一样**。

在继续下一章之前请花时间**仔细回顾并尝试额外挑战**。这对于稳固基础十分重要。

在下一单元，我们**将要学习 [Optuna](https://optuna.org/)**. 一个深度强化学习中最关键的任务，寻找最优超参数。 Optuna 是一个帮助你进行自动搜索的库。






### 这是一个与您一起构建的课程 👷🏿‍♀️

最后，我们希望通过您的反馈来迭代地改进和更新课程。如果您有任何反馈，请填写此表格👉https://forms.gle/3HgA7bEHwAmmLfwh9。

我们一直在努力改进我们的教程，所以如果您在这个 **Notebook 中发现任何问题**，请在 Github 仓库中[开一个 issue](https://github.com/huggingface/deep-rl-class/issues)。

让我们在奖励单元 2 再见! 🔥

### 保持热爱，奔赴山海 🤗